## Explorative analysis: FrameNet and ECG

## Usage

To use this notebook, you need my FrameNet package. I don't think you have `Git` installed on your machine, but you can download it from [here](https://github.com/lucag/framenet/archive/lucag.zip) 

and put in somewhere you can easily reach. The zip above expands to a folder `framenet-lucag` in a place that's easily reachable from the command line — yes, you need one!

* open a command line by either 

    - hitting `Win`-`S` on Windows 8 or `Win`-`R` on Windows 7
    - opening a "Windows Command Prompt" from the Start menu


* go under `framenet-lucag`    


* do (there's a space and a dot after `-e`)
    
```bash
pip3 install -e .
```
 
If you do have `Git`, clone the repo using my branch, like so:

```bash
git clone -b lucag https://github.com/lucag/framenet.git
```

then do:
    
```bash
cd framenet
pip3 install -e .
```

After that you should be able to run everyting in here.

In [1]:
import pandas as pd
import qgrid
import framenet.ecg.generation as gen

from itertools        import chain
from framenet.util    import aget, curry, flatten, take, drop, dfs_iterator, flatmap
from pprint           import pprint, pformat
from collections      import defaultdict
from framenet.builder import build
from functools        import reduce

fn, fnb = build()

# Test it's working!
msg = 'Something went wrong. Please check your FrameNet library'
assert fn.get_frame('Event').name == 'Event', msg

def diff(xs, ys):
    return [x for x in xs if x not in ys]

def incl(xs, ys):
    return [(x, x in ys) for x in xs]

fmt   = aget('name', 'coreType', 'frame', default='<unk>')
names = lambda es: [e.name for e in es]

@curry
def inherited(frame):
    return [r for r in frame.fe_relations 
            if r.subFrame == frame.name and r.name.startswith('Inh')]

@curry
def inherited_fes(fn, fers, frame):
    """Returns inherited FE IDs using FE relations.
    """
    return [(fer['supID'], fer['relationSupID']) 
            for fer in fers 
            if fer['relationType'] == 'Inheritance' 
               and fer['relationSubID'] == frame.ID]
    
ancestors = fn.typesystem.get_ancestors

def unique(xs):
    marked = set()
    ys = []
    for x in xs:
        if x not in marked:
            ys.append(x)
            marked.add(x)
    return ys

def inherited_elements(src_frames):
    return unique(flatten(p.elements for p in src_frames))

@curry
def fe_parents(fers, feID):
    return fers[fers.subID == feID] 

@curry
def fe_ancestors(fers, feID):
    return dfs_iterator(set(), fe_parents(fers), lambda fe: fe.ID, feID)
    
@curry
def inherited_elements2(fers, src_frames):
    pp = fe_parents(fers)
    return unique(flatten(pp(fe) for fe in src_frames))

ancestors     = fn.typesystem.get_ancestors

# def check(fn, i, frame):
#     inherited_es  = set(flatten(p.elements for p in ancestors(frame)))
#     elements      = set(frame.elements)
#     d1            = incl(inherited_es, elements)
#     not_in        = [(e.name, inherited(frame, e)) for e, in_ in d1 if not in_]
#     assert not not_in, '%s (%d):\n%s' % (frame.name, i, pformat(not_in))
    
# #     d2 = [fmt(e) for e in diff(ns, all_elements)]
# #     assert not d1, '%s [%d]: in ancestors, but not in frame: %s' % (
#             frame.name, i, pformat(d1))
# #     assert not d2, '%s [%d]: in frame, but not in ancestors: %s' % (
#             frame.name, i, pformat(d2))
#     return not not_in #and not d2

def invert(pairs):
    inv = defaultdict(list)
    for k, v in pairs: inv[v].append(k)
    return inv
    
def as_data_frame(felts):
    ATTRS = 'frame_name name coreType semtype requires excludes'.split()
    return pd.DataFrame([[getattr(fe, a) for a in ATTRS] for fe in felts], columns=ATTRS)

## Done!

1. Do this for `Cause_motion` (and `Fluidic_motion`)
1. `FE` <--> `FE` relations for `Transitive_action`, `Cause_motion`, `Fluidic_motion`, `Passing` 
1. ECG schema for `Cause_motion`
1. There is no 4th thing! :-D

### All the `FE`s for `Cause_motion` with their respective frame:

In [2]:
Cause_motion = fn.get_frame('Cause_motion')
inh_elts = inherited_elements(ancestors(Cause_motion))
print(len(inh_elts))
as_data_frame(inh_elts)

50


,frame_name,name,coreType,semtype,requires,excludes
0,Event,Event,Core-Unexpressed,State_of_affairs,[],[]
1,Event,Place,Core,Locative_relation,[],[]
2,Event,Explanation,Extra-Thematic,State_of_affairs,[],[]
3,Event,Time,Core,Time,[],[]
4,Event,Duration,Extra-Thematic,Duration,[],[]
5,Event,Manner,Peripheral,Manner,[],[]
6,Event,Frequency,Extra-Thematic,None,[],[]
7,Event,Timespan,Extra-Thematic,None,[],[]
8,Objective_influence,Influencing_entity,Core,None,[],[Influencing_variable]
9,Objective_influence,Dependent_entity,Core,None,[],[Dependent_variable]


### All the `FE`s for `Fluidic_motion` with their respective frame:

In [3]:
Fluidic_motion = fn.get_frame('Fluidic_motion')
Fm_ancestors   = ancestors(Fluidic_motion)
Fm_inh_elts    = inherited_elements(ancestors(Fluidic_motion))

print('Length:',    len(Fm_inh_elts))
# print('Ancestors:', list(Fm_ancestors))

as_data_frame(Fm_inh_elts)

Length: 44


,frame_name,name,coreType,semtype,requires,excludes
0,Event,Event,Core-Unexpressed,State_of_affairs,[],[]
1,Event,Place,Core,Locative_relation,[],[]
2,Event,Explanation,Extra-Thematic,State_of_affairs,[],[]
3,Event,Time,Core,Time,[],[]
4,Event,Duration,Extra-Thematic,Duration,[],[]
5,Event,Manner,Peripheral,Manner,[],[]
6,Event,Frequency,Extra-Thematic,None,[],[]
7,Event,Timespan,Extra-Thematic,None,[],[]
8,Motion,Theme,Core,Physical_object,[],[]
9,Motion,Source,Core,Source,[],[Area]


### All the `FE` relations for `Fluidic_motion` available in FrameNet:

In [5]:
Fluidic_motion_FE_relations = gen.fe_relations_for(fn, fn.get_frame('Fluidic_motion'))
# print(Fluidic_motion_FE_relations.count())
print(Fluidic_motion_FE_relations.count())
Fluidic_motion_FE_relations

relationSubFrameName      75
subFEName                 75
relationType              75
relationSuperFrameName    75
superFEName               75
dtype: int64


,relationSubFrameName,subFEName,relationType,relationSuperFrameName,superFEName
3323,Fluidic_motion,Area,Inheritance,Motion,Area
3324,Fluidic_motion,Depictive,Inheritance,Motion,Depictive
3325,Fluidic_motion,Distance,Inheritance,Motion,Distance
3326,Fluidic_motion,Duration,Inheritance,Motion,Duration
3327,Fluidic_motion,Goal,Inheritance,Motion,Goal
3328,Fluidic_motion,Manner,Inheritance,Motion,Manner
3329,Fluidic_motion,Path,Inheritance,Motion,Path
3330,Fluidic_motion,Place,Inheritance,Motion,Place
3331,Fluidic_motion,Result,Inheritance,Motion,Result
3332,Fluidic_motion,Source,Inheritance,Motion,Source


### Just the `Inheritance` relations for `Fluidic_motion`:

In [6]:
Fm_inh_FE_rels = Fluidic_motion_FE_relations[
    Fluidic_motion_FE_relations.relationType == 'Inheritance']
print('Count:', Fm_inh_FE_rels.count())
Fm_inh_FE_rels

Count: relationSubFrameName      19
subFEName                 19
relationType              19
relationSuperFrameName    19
superFEName               19
dtype: int64


,relationSubFrameName,subFEName,relationType,relationSuperFrameName,superFEName
3323,Fluidic_motion,Area,Inheritance,Motion,Area
3324,Fluidic_motion,Depictive,Inheritance,Motion,Depictive
3325,Fluidic_motion,Distance,Inheritance,Motion,Distance
3326,Fluidic_motion,Duration,Inheritance,Motion,Duration
3327,Fluidic_motion,Goal,Inheritance,Motion,Goal
3328,Fluidic_motion,Manner,Inheritance,Motion,Manner
3329,Fluidic_motion,Path,Inheritance,Motion,Path
3330,Fluidic_motion,Place,Inheritance,Motion,Place
3331,Fluidic_motion,Result,Inheritance,Motion,Result
3332,Fluidic_motion,Source,Inheritance,Motion,Source


### `Transitive_action` `FE`s

In [7]:
as_data_frame(fn.get_frame('Transitive_action').elements)

,frame_name,name,coreType,semtype,requires,excludes
0,Transitive_action,Agent,Core,Sentient,[],[Cause]
1,Transitive_action,Patient,Core,None,[],[]
2,Transitive_action,Event,Core-Unexpressed,State_of_affairs,[],[]
3,Transitive_action,Depictive,Extra-Thematic,None,[],[]
4,Transitive_action,Result,Extra-Thematic,None,[],[]
5,Transitive_action,Means,Peripheral,State_of_affairs,[],[]
6,Transitive_action,Manner,Peripheral,Manner,[],[]
7,Transitive_action,Time,Peripheral,Time,[],[]
8,Transitive_action,Place,Peripheral,Locative_relation,[],[]
9,Transitive_action,Cause,Core-Unexpressed,None,[],[Means]


### `Objective_influence` `FE`s

In [8]:
as_data_frame(fn.get_frame('Objective_influence').elements)

,frame_name,name,coreType,semtype,requires,excludes
0,Objective_influence,Influencing_entity,Core,None,[],[Influencing_variable]
1,Objective_influence,Dependent_entity,Core,None,[],[Dependent_variable]
2,Objective_influence,Influencing_variable,Core-Unexpressed,None,[],[Influencing_situation]
3,Objective_influence,Dependent_situation,Core-Unexpressed,None,[],[Dependent_entity]
4,Objective_influence,Dependent_variable,Core-Unexpressed,None,[],[Dependent_situation]
5,Objective_influence,Influencing_situation,Core-Unexpressed,None,[],[Influencing_entity]
6,Objective_influence,Degree,Core-Unexpressed,Degree,[],[]
7,Objective_influence,Time,Peripheral,Time,[],[]
8,Objective_influence,Place,Peripheral,Locative_relation,[],[]
9,Objective_influence,Manner,Peripheral,Manner,[],[]


In [ ]:
fe_relations_for(fn, fn.get_frame('Objective_influence'))

### `Event` `FE`s

In [9]:
as_data_frame(fn.get_frame('Event').elements)

,frame_name,name,coreType,semtype,requires,excludes
0,Event,Event,Core-Unexpressed,State_of_affairs,[],[]
1,Event,Place,Core,Locative_relation,[],[]
2,Event,Explanation,Extra-Thematic,State_of_affairs,[],[]
3,Event,Time,Core,Time,[],[]
4,Event,Duration,Extra-Thematic,Duration,[],[]
5,Event,Manner,Peripheral,Manner,[],[]
6,Event,Frequency,Extra-Thematic,None,[],[]
7,Event,Timespan,Extra-Thematic,None,[],[]


### All the FE Relations for  the `Cause_motion` frame.

In [11]:
# Cause_motion
gen.fe_relations_for(fn, Cause_motion)

,relationSubFrameName,subFEName,relationType,relationSuperFrameName,superFEName
1974,Transitive_action,Manner,Inheritance,Objective_influence,Manner
1975,Transitive_action,Place,Inheritance,Objective_influence,Place
1976,Transitive_action,Time,Inheritance,Objective_influence,Time
1977,Transitive_action,Agent,Inheritance,Objective_influence,Influencing_entity
1978,Transitive_action,Patient,Inheritance,Objective_influence,Dependent_entity
1979,Transitive_action,Cause,Inheritance,Objective_influence,Influencing_situation
2004,Cause_motion,Agent,Inheritance,Transitive_action,Agent
2005,Cause_motion,Cause,Inheritance,Transitive_action,Cause
2006,Cause_motion,Depictive,Inheritance,Transitive_action,Depictive
2007,Cause_motion,Manner,Inheritance,Transitive_action,Manner


### FE relations for `Transitive_action`:

In [12]:
gen.fe_relations_for(fn, fn.get_frame('Transitive_action'))

,relationSubFrameName,subFEName,relationType,relationSuperFrameName,superFEName
1974,Transitive_action,Manner,Inheritance,Objective_influence,Manner
1975,Transitive_action,Place,Inheritance,Objective_influence,Place
1976,Transitive_action,Time,Inheritance,Objective_influence,Time
1977,Transitive_action,Agent,Inheritance,Objective_influence,Influencing_entity
1978,Transitive_action,Patient,Inheritance,Objective_influence,Dependent_entity
1979,Transitive_action,Cause,Inheritance,Objective_influence,Influencing_situation
4044,Objective_influence,Manner,Inheritance,Event,Manner
4045,Objective_influence,Place,Inheritance,Event,Place
4046,Objective_influence,Time,Inheritance,Event,Time
5251,Event,Place,Subframe,Change_of_state_scenario,Place


## ECG Schema representations

### Schema for `Cause_motion`:

In [13]:
print(gen.format_schema(fn, Cause_motion))

schema Cause_motion 
    subcase of Transitive_action 
    evokes Motion as motion 
    roles 
       // agent: @sentient (inherited from Transitive_action)
       // cause (inherited from Transitive_action)
       // depictive (inherited from Transitive_action)
       // manner: @manner (inherited from Transitive_action)
       // means: @state_of_affairs (inherited from Transitive_action)
       // place: @locative_relation (inherited from Transitive_action)
       // result (inherited from Transitive_action)
       // time: @time (inherited from Transitive_action)
       area
       degree: @degree
       distance
       explanation
       goal: @goal
       handle
       initial_state
       instrument: @physical_entity
       path
       source: @source
       subregion: @locative_relation
       theme: @physical_object
     constraints 
       // inherited from Transitive_action: agent ⟷ agent
       // inherited from Transitive_action: cause ⟷ cause
       // inherited from Tran

### Schema for `Transitive_action`:

In [14]:
print(gen.format_schema(fn, fn.get_frame('Transitive_action')))

schema Transitive_action 
    subcase of Objective_influence 
    roles 
       // manner: @manner (inherited from Objective_influence)
       // place: @locative_relation (inherited from Objective_influence)
       // time: @time (inherited from Objective_influence)
       agent: @sentient
       cause
       depictive
       event: @state_of_affairs
       means: @state_of_affairs
       patient
       result
     constraints 
       // inherited from Objective_influence: manner ⟷ manner
       // inherited from Objective_influence: place ⟷ place
       // inherited from Objective_influence: time ⟷ time
       influencing_entity ⟷ agent
       dependent_entity ⟷ patient
       influencing_situation ⟷ cause


### `FE`s for `Objective_influence`:

In [15]:
Objective_influence = fn.get_frame('Objective_influence')
as_data_frame(Objective_influence.elements)

,frame_name,name,coreType,semtype,requires,excludes
0,Objective_influence,Influencing_entity,Core,None,[],[Influencing_variable]
1,Objective_influence,Dependent_entity,Core,None,[],[Dependent_variable]
2,Objective_influence,Influencing_variable,Core-Unexpressed,None,[],[Influencing_situation]
3,Objective_influence,Dependent_situation,Core-Unexpressed,None,[],[Dependent_entity]
4,Objective_influence,Dependent_variable,Core-Unexpressed,None,[],[Dependent_situation]
5,Objective_influence,Influencing_situation,Core-Unexpressed,None,[],[Influencing_entity]
6,Objective_influence,Degree,Core-Unexpressed,Degree,[],[]
7,Objective_influence,Time,Peripheral,Time,[],[]
8,Objective_influence,Place,Peripheral,Locative_relation,[],[]
9,Objective_influence,Manner,Peripheral,Manner,[],[]


Here is the ECG schema for `Objective_influence`: the exclusion pattern above (for the `dependent_*` and `influencing_*` `FE`s are not expressed.

In [16]:
print(gen.format_schema(fn, Objective_influence))

schema Objective_influence 
    subcase of Event 
    roles 
       // manner: @manner (inherited from Event)
       // place: @locative_relation (inherited from Event)
       // time: @time (inherited from Event)
       circumstances
       degree: @degree
       dependent_entity
       dependent_situation
       dependent_variable
       descriptor
       influencing_entity
       influencing_situation
       influencing_variable
     constraints 
       // inherited from Event: manner ⟷ manner
       // inherited from Event: place ⟷ place
       // inherited from Event: time ⟷ time


### Schema for `Event`:

In [17]:
print(gen.format_schema(fn, fn.get_frame('Event')))

schema Event 
    roles 
       duration: @duration
       event: @state_of_affairs
       explanation: @state_of_affairs
       frequency
       manner: @manner
       place: @locative_relation
       time: @time
       timespan
     constraints 
       place ⟷ place
       time ⟷ time
       event ⟷ event
       explanation ⟷ explanation


Here are the `FE`s for `Event`:

In [18]:
as_data_frame(fn.get_frame('Event').elements)

,frame_name,name,coreType,semtype,requires,excludes
0,Event,Event,Core-Unexpressed,State_of_affairs,[],[]
1,Event,Place,Core,Locative_relation,[],[]
2,Event,Explanation,Extra-Thematic,State_of_affairs,[],[]
3,Event,Time,Core,Time,[],[]
4,Event,Duration,Extra-Thematic,Duration,[],[]
5,Event,Manner,Peripheral,Manner,[],[]
6,Event,Frequency,Extra-Thematic,None,[],[]
7,Event,Timespan,Extra-Thematic,None,[],[]


## `Cause_fluidic_motion`

In [19]:
Cause_fluidic_motion = fn.get_frame('Cause_fluidic_motion')
as_data_frame(Cause_fluidic_motion.elements)

,frame_name,name,coreType,semtype,requires,excludes
0,Cause_fluidic_motion,Agent,Core,Sentient,[],[Cause]
1,Cause_fluidic_motion,Cause,Core,None,[],[]
2,Cause_fluidic_motion,Fluid,Core,None,[],[]
3,Cause_fluidic_motion,Source,Core,Source,[],[Area]
4,Cause_fluidic_motion,Goal,Core,Goal,[],[Area]
5,Cause_fluidic_motion,Path,Core,None,[],[Area]
6,Cause_fluidic_motion,Area,Core,None,[],[]
7,Cause_fluidic_motion,Manner,Peripheral,Manner,[],[]
8,Cause_fluidic_motion,Time,Peripheral,Time,[],[]
9,Cause_fluidic_motion,Explanation,Extra-Thematic,State_of_affairs,[],[]


In [20]:
print(gen.format_schema(fn, Cause_fluidic_motion))

schema Cause_fluidic_motion 
    subcase of Cause_motion 
    evokes Fluidic_motion as fluidic_motion 
    roles 
       // agent: @sentient (inherited from Cause_motion)
       // area (inherited from Cause_motion)
       // cause (inherited from Cause_motion)
       // distance (inherited from Cause_motion)
       // explanation: @state_of_affairs (inherited from Cause_motion)
       // goal: @goal (inherited from Cause_motion)
       // instrument: @physical_entity (inherited from Cause_motion)
       // manner: @manner (inherited from Cause_motion)
       // means: @state_of_affairs (inherited from Cause_motion)
       // path (inherited from Cause_motion)
       // place: @locative_relation (inherited from Cause_motion)
       // result (inherited from Cause_motion)
       // source: @source (inherited from Cause_motion)
       // time: @time (inherited from Cause_motion)
       fluid
       purpose: @state_of_affairs
     constraints 
       // inherited from Cause_motion: agent 

**NOTE**: despite the roles above are marked as `Inherited`, they show up in the constraint secion anyway. The reason is that they also are marked as `ReFraming_Mapping` and `Subframe`: 

In [21]:
gen.fe_relations_for(fn, Cause_fluidic_motion)

,relationSubFrameName,subFEName,relationType,relationSuperFrameName,superFEName
1683,Cause_fluidic_motion,Agent,Inheritance,Cause_motion,Agent
1684,Cause_fluidic_motion,Area,Inheritance,Cause_motion,Area
1685,Cause_fluidic_motion,Cause,Inheritance,Cause_motion,Cause
1686,Cause_fluidic_motion,Distance,Inheritance,Cause_motion,Distance
1687,Cause_fluidic_motion,Goal,Inheritance,Cause_motion,Goal
1688,Cause_fluidic_motion,Instrument,Inheritance,Cause_motion,Instrument
1689,Cause_fluidic_motion,Manner,Inheritance,Cause_motion,Manner
1690,Cause_fluidic_motion,Means,Inheritance,Cause_motion,Means
1691,Cause_fluidic_motion,Path,Inheritance,Cause_motion,Path
1692,Cause_fluidic_motion,Place,Inheritance,Cause_motion,Place
